In [1]:
from typing import List, Tuple, Dict, Any, Optional
from pandarallel import pandarallel
import os
import re
import sys
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
import dspy
import google.generativeai as genai
GEMINI_TOKEN = Path("..", "gemini_token.txt").resolve().read_text().strip()
genai.configure(api_key=GEMINI_TOKEN)

# Read the groq token
GROQ_TOKEN = Path("../groq_token.txt").read_text().strip()
os.environ["GROQ_API_KEY"] = GROQ_TOKEN

lm = dspy.LM('groq/llama-3.3-70b-versatile')
dspy.configure(lm=lm)


FOLDER_WITH_API_MIGRATION_KNOWLEDGE = Path(
    "..", "data/changelogs/v001/api_migration_docs")

In [3]:
# read all docs in the folder
model = genai.GenerativeModel("models/gemini-1.5-flash")

docs = []
for file in FOLDER_WITH_API_MIGRATION_KNOWLEDGE.glob("*.md"):
    doc = Path(file).read_text()
    docs.append(doc)
    response = model.count_tokens(doc)
    total_tokens = response.total_tokens
    print(f"Read {file} > {total_tokens} tokens")

Read ../data/changelogs/v001/api_migration_docs/1.0_1.1_1.2.md > 61572 tokens
Read ../data/changelogs/v001/api_migration_docs/packaging_changes.md > 8119 tokens
Read ../data/changelogs/v001/api_migration_docs/features_changes.md > 7452 tokens


In [13]:
response = model.generate_content(
    f"who are you in one sentence? {docs[1]} mention also one random sentence from the doc above")
print(response.text)

I am a large language model, providing a summary of Qiskit 1.0 packaging changes and troubleshooting advice.



In [14]:
model = genai.GenerativeModel("gemini-1.5-flash")
instruction = "Given this documentation about the API migration, upgrade the given code snippet to use qiskit v1. reply only with code."
prompt = f"""
{docs[0]}
{docs[1]}
{docs[2]}
{instruction}
```python
from qiskit import QuantumCircuit, execute, Aer

# Create a quantum circuit with 3 qubits and 3 classical bits
qc = QuantumCircuit(3, 3)

# Prepare a hidden state using CX (CNOT) gates and a Toffoli gate
qc.x(0)
qc.ccx(0, 1, 2)

# Apply a mid-circuit measurement
qc.measure([2], [2])

# Use the measurement outcome to conditionally apply a CX gate
qc.cx(1, 0).c_if(2, 1)

# Measure the final state
qc.measure([0, 1], [0, 1])

# Run the circuit on a simulator
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)
result = job.result()

# Print the outcome probabilities
counts = result.get_counts(qc)
print(counts)
```
"""
response = model.generate_content(prompt)
print(response.text)

KeyboardInterrupt: 

# Generate Creativityy Seed

In [25]:
import requests
from bs4 import BeautifulSoup


def fetch_markdown_from_url(url: str) -> str:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    else:
        raise Exception(
            f"Failed to retrieve the page. Status code: {response.status_code}")


def get_stackoverflow_inspiration() -> str:
    url = "https://www.isimonbrown.co.uk/dicestack/?site=stackoverflow&min_score=10&tagged=&answer_status=any"
    page_content = fetch_markdown_from_url(url)
    start_index = page_content.find("Answer status")
    if start_index != -1:
        page_content = page_content[start_index:]
        lines = page_content.split('\n')
        lines = lines[3:-20]
        cleaned_lines = [re.sub(r'^\d+', '', line) for line in lines]
        # remove empty lines
        cleaned_lines = [line for line in cleaned_lines if line.strip()]
        page_content = '\n'.join(cleaned_lines)
    return page_content


# Example usage
markdown = get_stackoverflow_inspiration()
print(markdown)

How to install &quot;ifconfig&quot; command in my ubuntu docker image?
Fastest way to check whether a value exists more often than X in a list
How to access People album used in Apple&#39;s Photos app
Dictionary of lists to dataframe
NumPy version of &quot;Exponential weighted moving average&quot;, equivalent to pandas.ewm().mean()
java.lang.NoClassDefFoundError $$inlined$forEach$lambda$1 in Kotlin
How to set the height for a flexbox container so it uses the &quot;remaining space&quot;
How to specify &quot;sources&quot; JAR for local JAR dependency?
Treat all warnings as errors, except some
How to access data from computed property Vue.js


In [26]:
inspiring_questions_titles = get_stackoverflow_inspiration()


class BrainstormQiskitTitles(dspy.Signature):
    """Generate fictional titles for questions on quantum computing using qiskit v0.45."""

    inspiring_titles: str = dspy.InputField(
        desc="Inspiring titles from StackOverflow")
    num_titles: int = dspy.InputField(desc="Number of titles to generate")
    generated_titles: List[str] = dspy.OutputField(
        desc="List of generated titles")


qiskit_titles_generator = dspy.ChainOfThought(BrainstormQiskitTitles)
response = qiskit_titles_generator(
    inspiring_titles=inspiring_questions_titles,
    num_titles=20
)
print(response.generated_titles)

['How to implement quantum teleportation using Qiskit v0.45?', 'Understanding the difference between quantum gates and classical gates in Qiskit', "Optimizing quantum circuit depth using Qiskit's transpile function", 'Quantum error correction: How to implement surface codes using Qiskit', "Visualizing quantum states using Qiskit's Bloch sphere representation", 'Can I use Qiskit to simulate quantum many-body systems?', 'Implementing quantum algorithms for machine learning using Qiskit', 'Quantum circuit synthesis: How to convert a classical circuit to a quantum circuit using Qiskit', 'Using Qiskit to study quantum entanglement and non-locality', "Quantum simulation of chemical reactions using Qiskit's chemistry module", 'Qiskit vs. Cirq: Which quantum development framework is right for me?', 'Quantum computing with Qiskit: How to get started with quantum programming', 'Quantum machine learning with Qiskit: A tutorial on quantum k-means', "Quantum error mitigation techniques using Qiskit

In [28]:
titles = response.generated_titles
for title in titles:
    print(title)

How to implement quantum teleportation using Qiskit v0.45?
Understanding the difference between quantum gates and classical gates in Qiskit
Optimizing quantum circuit depth using Qiskit's transpile function
Quantum error correction: How to implement surface codes using Qiskit
Visualizing quantum states using Qiskit's Bloch sphere representation
Can I use Qiskit to simulate quantum many-body systems?
Implementing quantum algorithms for machine learning using Qiskit
Quantum circuit synthesis: How to convert a classical circuit to a quantum circuit using Qiskit
Using Qiskit to study quantum entanglement and non-locality
Quantum simulation of chemical reactions using Qiskit's chemistry module
Qiskit vs. Cirq: Which quantum development framework is right for me?
Quantum computing with Qiskit: How to get started with quantum programming
Quantum machine learning with Qiskit: A tutorial on quantum k-means
Quantum error mitigation techniques using Qiskit's Ignis module
Quantum circuit optimizat

## Diverse Titles

In [29]:
from sentence_transformers import SentenceTransformer
import numpy as np
import fpsample

# Load a pretrained Sentence Transformer model
EMBEDDING_MODEL = "multi-qa-mpnet-base-cos-v1"
model = SentenceTransformer(EMBEDDING_MODEL)

# Convert the generated titles to embeddings
titles_embeddings = model.encode(titles, batch_size=32, show_progress_bar=True)

# Perform FPS sampling to get the 3 most far apart titles
fps_samples_idx = fpsample.fps_sampling(titles_embeddings, 3)

# Get the sampled titles
sampled_titles = [titles[idx] for idx in fps_samples_idx]
for title in sampled_titles:
    print(title)

/home/paltenmo/.conda/envs/crosspl/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Quantum computing with Qiskit: A survey of quantum algorithms and techniques
Visualizing quantum states using Qiskit's Bloch sphere representation
Optimizing quantum circuit depth using Qiskit's transpile function


## Get Task Descriptions

In [27]:
class ExpandQiskitTitleToDescription(dspy.Signature):
    """Expand a title to a full description of a question on Stack Overflow about Qiskit."""

    title: str = dspy.InputField(desc="Title of the question")
    expanded_description: str = dspy.OutputField(
        desc="Expanded description of the question")


expand_qiskit_title_to_description = dspy.ChainOfThought(
    ExpandQiskitTitleToDescription)

response = expand_qiskit_title_to_description(
    title="How to implement quantum error correction in Qiskit v0.45"
)
print(response.expanded_description)

I am trying to implement quantum error correction using Qiskit version 0.45. Specifically, I want to know how to use the tools and libraries provided by Qiskit to protect my quantum circuits from errors due to decoherence and other noise. I have a basic understanding of quantum error correction codes such as the surface code, Shor's code, and quantum convolutional codes, but I need guidance on how to apply these codes in Qiskit. I would like to know the steps involved in encoding quantum information, applying error correction, and decoding the information. Additionally, I would appreciate any examples or tutorials that demonstrate how to use Qiskit's built-in functions for quantum error correction.


## Implementation

In [4]:

class ImplementQiskitAnswer(dspy.Signature):
    """Implement a Qiskit answer in Qiskit v0.45 and return only the Python code snippet."""

    title: str = dspy.InputField(desc="Title of the question")
    expanded_description: str = dspy.InputField(
        desc="Expanded description of the question")
    code_snippet: str = dspy.OutputField(
        desc="Self-contained Python code snippet for the implementation")


implement_qiskit_answer = dspy.ChainOfThought(ImplementQiskitAnswer)

response = implement_qiskit_answer(
    title="How to implement quantum error correction in Qiskit v0.45",
    expanded_description="I am trying to implement quantum error correction using Qiskit version 0.45. Specifically, I want to know how to use the tools and libraries provided by Qiskit to protect my quantum circuits from errors due to decoherence and other noise. I have a basic understanding of quantum error correction codes such as the surface code, Shor's code, and quantum convolutional codes, but I need guidance on how to apply these codes in Qiskit. I would like to know the steps involved in encoding quantum information, applying error correction, and decoding the information. Additionally, I would appreciate any examples or tutorials that demonstrate how to use Qiskit's built-in functions for quantum error correction."
)
print(response.code_snippet)

```python
from qiskit import QuantumCircuit, execute, Aer
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import AerSimulator

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Encode the quantum information using a simple repetition code
qc.x(0)  # encode |1
qc.cx(0, 1)
qc.cx(0, 2)

# Apply an error to the first qubit
qc.x(0).c_if(1, 0)  # conditional X gate to simulate an error

# Decode the quantum information
qc.cx(0, 1)
qc.cx(0, 2)
qc.x(0)

# Measure the qubits
qc.measure_all()

# Run the circuit on a simulator
simulator = AerSimulator()
job = execute(qc, simulator)
result = job.result()
counts = result.get_counts(qc)

print(counts)
```


# Upgrade to v1 Qiskit

In [7]:

def upgrade_qiskit_code(migration_documents: List[str], snippet: str) -> str:
    assert len(
        migration_documents) > 0, "There should be at least one migration document."
    model = genai.GenerativeModel("gemini-1.5-flash")
    instruction = "Given this documentation about the API migration, upgrade the given code snippet to use qiskit v1. reply only with code."
    prompt = "\n".join(
        migration_documents[:3]) + f"\n{instruction}\n```python\n{snippet}\n```"
    response = model.generate_content(prompt)
    return response.text


# Example usage
upgraded_code = upgrade_qiskit_code(
    migration_documents=docs,
    snippet=response.code_snippet
)
print(upgraded_code)

```python
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.quantum_info import Statevector

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Encode the quantum information using a simple repetition code
qc.x(0)  # encode |1
qc.cx(0, 1)
qc.cx(0, 2)

# Apply an error to the first qubit
qc.x(0).c_if(1, 0)  # conditional X gate to simulate an error

# Decode the quantum information
qc.cx(0, 1)
qc.cx(0, 2)
qc.x(0)

# Measure the qubits
qc.measure_all()

# Run the circuit on a simulator
simulator = BasicSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc)
result = job.result()
counts = result.get_counts(qc)

print(counts)

#Alternative using Statevector (no measurement)
qc_no_meas = qc.copy()
qc_no_meas.remove_final_measurements()
statevector = Statevector(qc_no_meas)
print(statevector)
```



# Validate via Run

In [16]:
import docker
import tempfile


def remove_backticks_lines(text: str) -> str:
    new_lines = [
        line for line in text.split('\n') if not line.startswith('```')]
    return '\n'.join(new_lines)


def run_qiskit_code_in_docker(code: str, image_name: str) -> str:
    """
    Executes a given Qiskit code snippet inside a Docker container.
    Args:
        code (str): The Qiskit code to be executed.
        image_name (str): The name of the Docker image to use for execution.
    Returns:
        str: The output logs from the Docker container execution.
    Raises:
        Exception: If there is an error during the Docker container execution.
    Example:
        code = '''
        from qiskit import QuantumCircuit, transpile, Aer, execute
        qc = QuantumCircuit(2)
        qc.h(0)
        qc.cx(0, 1)
        qc.measure_all()
        backend = Aer.get_backend('qasm_simulator')
        result = execute(qc, backend, shots=1024).result()
        counts = result.get_counts()
        print(counts)
        '''
        image_name = 'qiskit/qiskit:latest'
        output = run_qiskit_code_in_docker(code, image_name)
        print(output)
    """

    # Create a temporary file for the code snippet
    with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as code_file:
        code_file.write(remove_backticks_lines(code).encode())
        code_file_path = code_file.name

    client = docker.from_env()

    try:
        # Execute the Docker command
        container = client.containers.run(
            image_name,
            f"python /workspace/code_sample.py",
            volumes={code_file_path: {'bind': '/workspace/code_sample.py', 'mode': 'rw'}},
            detach=True
        )

        # Wait for the container to finish
        container.wait()

        # Get the logs
        logs = container.logs().decode('utf-8')
        return logs

    except Exception as e:
        return str(e)

    finally:
        # Clean up
        container.remove()
        code_file.close()


# Example usage
old_version_logs = run_qiskit_code_in_docker(
    response.code_snippet, "qiskit_image_0.45.0")
new_version_logs = run_qiskit_code_in_docker(
    upgraded_code, "qiskit_image_1.2.0")

print("Old Version Logs:")
print(old_version_logs)
print("New Version Logs:")
print(new_version_logs)

Old Version Logs:
Traceback (most recent call last):
  File "/workspace/code_sample.py", line 14, in <module>
    qc.x(0).c_if(1, 0)  # conditional X gate to simulate an error
  File "/usr/local/lib/python3.10/site-packages/qiskit/circuit/instructionset.py", line 133, in c_if
    classical = self._requester(classical)
  File "/usr/local/lib/python3.10/site-packages/qiskit/circuit/quantumcircuit.py", line 1200, in _resolve_classical_resource
    raise CircuitError(f"Classical bit index {specifier} is out-of-range.") from None
qiskit.circuit.exceptions.CircuitError: 'Classical bit index 1 is out-of-range.'

New Version Logs:
Traceback (most recent call last):
  File "/workspace/code_sample.py", line 14, in <module>
    qc.x(0).c_if(1, 0)  # conditional X gate to simulate an error
  File "/usr/local/lib/python3.10/site-packages/qiskit/circuit/instructionset.py", line 142, in c_if
    classical = self._requester(classical)
  File "/usr/local/lib/python3.10/site-packages/qiskit/circuit/quan

# Auto-Fixer

In [ ]:

class ChipInAndHelp(dspy.Signature):
    """Help another user with a wrong answer by providing a corrected implementation."""

    title: str = dspy.InputField(desc="Title of the question")
    description: str = dspy.InputField(desc="Description of the question")
    implementation: str = dspy.InputField(
        desc="Original implementation code snippet")
    log_output: str = dspy.InputField(
        desc="Log output of the original implementation")
    corrected_code: str = dspy.OutputField(
        desc="Self-contained corrected Python code snippet")


chip_in_and_help = dspy.ChainOfThought(ChipInAndHelp)

response = chip_in_and_help(
    title="How to implement quantum error correction in Qiskit v0.45",
    description="I am trying to implement quantum error correction using Qiskit version 0.45. Specifically, I want to know how to use the tools and libraries provided by Qiskit to protect my quantum circuits from errors due to decoherence and other noise. I have a basic understanding of quantum error correction codes such as the surface code, Shor's code, and quantum convolutional codes, but I need guidance on how to apply these codes in Qiskit. I would like to know the steps involved in encoding quantum information, applying error correction, and decoding the information. Additionally, I would appreciate any examples or tutorials that demonstrate how to use Qiskit's built-in functions for quantum error correction.",
    implementation='''from qiskit import QuantumCircuit, execute, Aer

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Encode the quantum information using a simple repetition code
qc.x(0)  # encode |1
qc.cx(0, 1)
qc.cx(0, 2)

# Apply an error to the first qubit
qc.x(0).c_if(1, 0)  # conditional X gate to simulate an error

# Decode the quantum information
qc.cx(0, 1)
qc.cx(0, 2)
qc.x(0)

# Measure the qubits
qc.measure_all()

# Run the circuit on a simulator
simulator = AerSimulator()
job = execute(qc, simulator)
result = job.result()
counts = result.get_counts(qc)

print(counts)''',
    log_output="Traceback (most recent call last):\n  File \"/workspace/code_sample.py\", line 14, in <module>\n    qc.x(0).c_if(1, 0)  # conditional X gate to simulate an error\n  File \"/usr/local/lib/python3.10/site-packages/qiskit/circuit/instructionset.py\", line 142, in c_if\n    classical = self._requester(classical)\n  File \"/usr/local/lib/python3.10/site-packages/qiskit/circuit/quantumcircuit.py\", line 6630, in resolve_classical_resource\n    raise CircuitError(f\"Classical bit index {specifier} is out-of-range.\") from None\nqiskit.circuit.exceptions.CircuitError: 'Classical bit index 1 is out-of-range.'"
)

print(response.corrected_code)

```python
from qiskit import QuantumCircuit, execute, Aer
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Encode the quantum information using a simple repetition code
qc.x(0)  # encode |1
qc.cx(0, 1)
qc.cx(0, 2)

# Define an error model
error_model = NoiseModel()
error_model.add_all_qubit_quantum_error(depolarizing_error(0.1, 1), 'x')

# Run the circuit on a simulator with the error model
simulator = AerSimulator(noise_model=error_model)
job = execute(qc, simulator)
result = job.result()
counts = result.get_counts(qc)

print(counts)
```


In [20]:
old_version_logs = run_qiskit_code_in_docker(
    response.corrected_code, "qiskit_image_0.45.0")
print(old_version_logs)

Traceback (most recent call last):
  File "/workspace/code_sample.py", line 22, in <module>
    counts = result.get_counts(qc)
  File "/usr/local/lib/python3.10/site-packages/qiskit/result/result.py", line 289, in get_counts
    raise QiskitError(f'No counts for experiment "{repr(key)}"')
qiskit.exceptions.QiskitError: 'No counts for experiment "<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f57db176b90>"'



In [21]:
response = chip_in_and_help(
    title="How to implement quantum error correction in Qiskit v0.45",
    description="I am trying to implement quantum error correction using Qiskit version 0.45. Specifically, I want to know how to use the tools and libraries provided by Qiskit to protect my quantum circuits from errors due to decoherence and other noise. I have a basic understanding of quantum error correction codes such as the surface code, Shor's code, and quantum convolutional codes, but I need guidance on how to apply these codes in Qiskit. I would like to know the steps involved in encoding quantum information, applying error correction, and decoding the information. Additionally, I would appreciate any examples or tutorials that demonstrate how to use Qiskit's built-in functions for quantum error correction.",
    implementation='''
from qiskit import QuantumCircuit, execute, Aer
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

# Create a quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# Encode the quantum information using a simple repetition code
qc.x(0)  # encode |1
qc.cx(0, 1)
qc.cx(0, 2)

# Define an error model
error_model = NoiseModel()
error_model.add_all_qubit_quantum_error(depolarizing_error(0.1, 1), 'x')

# Run the circuit on a simulator with the error model
simulator = AerSimulator(noise_model=error_model)
job = execute(qc, simulator)
result = job.result()
counts = result.get_counts(qc)

print(counts)
''',
    log_output="Traceback (most recent call last):\n  File \"/workspace/code_sample.py\", line 22, in <module>\n    counts = result.get_counts(qc)\n  File \"/usr/local/lib/python3.10/site-packages/qiskit/result/result.py\", line 289, in get_counts\n    raise QiskitError(f'No counts for experiment \"{repr(key)}\"')\nqiskit.exceptions.QiskitError: 'No counts for experiment \"<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f57db176b90>\"'"
)

old_version_logs = run_qiskit_code_in_docker(
    response.corrected_code, "qiskit_image_0.45.0")
print(old_version_logs)

{'000': 55, '111': 969}



## Get Extra Dependencies

In [32]:
class GetPipPackages(dspy.Signature):
    """Given a program snippet, return the names of the packages to pip install to make sure it can run, assume that Qiskit is already installed."""

    program_snippet: str = dspy.InputField(
        desc="The program snippet to analyze")
    package_names: List[str] = dspy.OutputField(
        desc="List of package names to pip install, empty if only Qiskit and matplotlib are needed.")


get_pip_packages = dspy.ChainOfThought(GetPipPackages)

response = get_pip_packages(
    program_snippet="""
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt

# Generate some random data for EDA
data = np.random.randn(100)

# Create a simple plot
sns.histplot(data, kde=True)
plt.title('Distribution of Random Data')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

"""
)
print(response.package_names)


response = get_pip_packages(
    program_snippet="""
from qiskit import QuantumCircuit, transpile, Aer, execute

# Create a quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate on qubit 0
qc.h(0)

# Add a CNOT gate on control qubit 0 and target qubit 1
qc.cx(0, 1)

# Measure both qubits
qc.measure_all()

# Transpile the circuit for the simulator backend
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend)

# Execute the circuit on the qasm simulator
job = execute(transpiled_qc, backend, shots=1024)
result = job.result()

# Get the counts (measurement results)
    counts = result.get_counts(qc)
    print(counts)
""",
)
print(response.package_names)

['numpy', 'scikit-learn', 'seaborn', 'matplotlib']
[]
